# Goal - Trying to find the underlying information in CIC data

In [1]:
# import packages
# Import important packages
import pandas as pd
import numpy as np
from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

## Read the Disbursed Loans from Loan Master table

In [2]:
sq = """select * from `risk_credit_mis.loan_master_table` where startApplyDateTime > '2021-12-31' and flagDisbursement = 1;"""

loandf = client.query(sq).to_dataframe()

### Check the shape of the data

In [3]:
print("The rows and columns in loan master table only disbursed loans are: ", loandf.shape)

The rows and columns in loan master table only disbursed loans are:  (54429, 224)


In [4]:
loandf.columns.values

array(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'loanRequestAmount', 'loanRequestTenure', 'startApplyDateTime',
       'customerCategory', 'applicationStatus', 'timeToDecision',
       'approvedDateTime', 'flagApproval', 'approvedLoanAmount',
       'approvedLoanTenure', 'rejectedDateTime', 'flagRejection',
       'signedDateTime', 'timeToSigned', 'flagSigned',
       'disbursementDateTime', 'payrollLinked', 'rateOfInterest',
       'flagDisbursement', 'disbursedLoanAmount', 'disbursedLoanTenur',
       'firstDueDate', 'secondDueDate', 'thirdDueDate', 'obsFPD00',
       'obsFPD05', 'obsFPD10', 'obsFPD15', 'obsFPD30', 'obsSPD10',
       'obsSPD15', 'obsSPD30', 'obsTPD10', 'obsTPD15', 'obsTPD30',
       'defFPD00', 'defFPD05', 'defFPD10', 'defFPD15', 'defFPD30',
       'defSPD10', 'defSPD15', 'defSPD30', 'defTPD10', 'defTPD15',
       'defTPD30', 'currentDelinquency', 'bucket', 'loanPaidStatus',
       'lastPaidDT', 'paymentDueButNotpaidDates', 'purposeDescriptio

In [5]:
loandf.to_csv("loanmastertabledisburseddata.csv", index = False)

## Read Loans which have Ever dpd more than 30

In [6]:
sq = """select loanAccountNumber, max(Max_ever_DPD) maxeverdpd from `risk_credit_mis.loan_bucket_flow_report_core`
group by 1
having  max(Max_ever_DPD) >= 30
;"""

loaneverdpd30 = client.query(sq).to_dataframe()

In [7]:
print("The rows and columns for loans accounts which ever touched 30 dpd or higher are: ", loaneverdpd30.shape)

The rows and columns for loans accounts which ever touched 30 dpd or higher are:  (36925, 2)


In [8]:
loaneverdpd30.to_csv("loanidtoughedever30dpd.csv", index = False)

## Read the cic summary table data

In [9]:
sq = """select 
* 
from `risk_credit_cic_data.cic_summary`   
;"""
cicdf = client.query(sq).to_dataframe()

In [10]:
print("The rows and columns in CIC data are: ", cicdf.shape)

The rows and columns in CIC data are:  (125153, 93)


In [11]:
cicdf.columns.values

array(['digitalLoanAccountId', 'crifApplicationId', 'customerId',
       'processEngineGuid', 'requestGuid', 'run_date', 'DetailID',
       'responseMsg', 'DescisionValue', 'DescisionDescription',
       'ExclusionRuleCode', 'exclusionRuleLabelValue',
       'exclusionRuleLabelDescription', 'ScoreRaw', 'ScoreRange',
       'Accounts30Days', 'Accounts30DaysOther', 'Accounts60Days',
       'Accounts60DaysOther', 'Accounts90Days', 'Accounts90DaysOther',
       'AgeAtLoanTermination', 'AgeAtLoanTerminationMax',
       'CreditAvgCreditLimit', 'CreditHighestCreditLimit',
       'CreditMaxOutstandingBalance', 'CreditMaxOutstandingBalanceDate',
       'CreditMaxOverdueAmount', 'CreditMaxOverdueAmountDate',
       'CreditMaxOverdueDays', 'CreditMaxOverdueDaysDate',
       'CreditNumberOfContracts', 'CreditTotalCreditLimit',
       'CreditTotalOutstandingBalanceAmount', 'CreditTotalOverdueAmount',
       'DataFromTest', 'InstMaxOverdueAmount', 'InstMaxOverdueAmountDate',
       'InstMaxOverdueDa

In [12]:
cicdf.sample(5)

,digitalLoanAccountId,crifApplicationId,customerId,processEngineGuid,requestGuid,run_date,DetailID,responseMsg,DescisionValue,DescisionDescription,...,UtilityTotalBilledAmount,UtilityTotalOutstandingBalanceAmount,UtilityTotalOverdueAmount,UtilizationRateCreditCard,UtilizationRateCreditCardOther,UtilizationRateNonInstallment,UtilizationRateNonInstallmentOther,label,scoreLabelDesc,scoreLabelValue
70733,a73e7f50-d887-4174-b509-5a06f52f4dd0,tonik-tul-192386,1477911,9b921b70-82a2-11ed-a66b-0242ace60012,9b70b0c0-82a2-11ed-8206-0242ace60003,2022-12-23,32777638638,None,Reject,,...,0,0,0,2,2,0,0,"[{""description"":""High Risk"",""value"":""HR""}, {""d...",High Risk,HR
55972,69a97276-aea3-423d-b2ac-16f87e1a2466,tonik-tul-238630,1412576,15cbb0f0-d121-11ed-a91e-0242ace60011,15b14b20-d121-11ed-b53a-0242ace60002,2023-04-02,32801542546,None,Review,,...,0,0,0,0,0,0,0,"[{""description"":""High Risk"",""value"":""HR""}, {""d...",Medium Risk,MR
93520,8db50799-3b89-4bd2-8683-1d74497f9288,tonik-sbcl-226258,1863263,3a345960-bf33-11ed-a91e-0242ace60011,3a19cc80-bf33-11ed-9ce1-0242ace60002,2023-03-10,32796063406,None,Review,,...,0,0,0,0,0,0,0,"[{""description"":""High Risk"",""value"":""HR""}, {""d...",None,None
85933,841306c0-a766-447e-86d2-36bd78df3d72,tonik-tul-257101,1031878,fd1d3850-eb13-11ed-b939-0242ace60015,fcfbcda0-eb13-11ed-be42-0242ace60004,2023-05-05,32810508958,None,Reject,,...,0,0,0,0,0,0,0,"[{""description"":""High Risk"",""value"":""HR""}, {""d...",High Risk,HR
29646,b5c8897b-56f9-46ab-a7cc-f5920bd561c1,tonik-tul-247339,1986454,69313810-dd32-11ed-9b82-0242ace60013,69180ac0-dd32-11ed-bac9-0242ace60006,2023-04-17,32805621366,1-098 - MORE SUBJECTS MATCHED,None,None,...,None,None,None,None,None,None,None,[],None,None


In [13]:
cicdf.to_csv("cicdata.csv", index = False)